In [1]:
from perses.app.relative_setup import RelativeFEPSetup
import simtk.unit as unit
from perses.annihilation.relative import HybridTopologyFactory
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import mcmc, utils
import openmmtools.integrators as integrators
import numpy as np

In [2]:
from openmmtools.utils import get_available_platforms
plats = get_available_platforms()
for plat in plats:
    print(plat.getName())
    
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
print(cache.global_context_cache.platform.getName())

Reference
CPU
conducting subsequent work with the following platform: CPU
CPU


In [3]:
molecule_setup = RelativeFEPSetup(ligand_input = 'test.smi', 
                                  old_ligand_index = 3, 
                                  new_ligand_index = 4, 
                                  forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml'], 
                                  phases = ['solvent'],
                                  protein_pdb_filename=None,
                                  receptor_mol2_filename=None, 
                                  pressure=1.0 * unit.atmosphere,
                                  temperature=300.0 * unit.kelvin, 
                                  solvent_padding=9.0 * unit.angstroms, 
                                  atom_map=None,
                                  hmass=4*unit.amus, 
                                  neglect_angles=False, 
                                  map_strength='default', 
                                  anneal_14s = False)

INFO:relative_setup:			_hmass: 4 Da.

INFO:relative_setup:Handling files for ligands and indices...
INFO:relative_setup:Detected .smi format.  Proceeding...
INFO:relative_setup:	old smiles: CCCO
INFO:relative_setup:	new smiles: CCCS
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:utils.openeye:molecule  does not have unique atom names. Generating now...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:relative_setup:	successfully created old and new systems from smiles
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:propan-1-ol
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 1.612347232708971), ('CustomTorsionForce', 0.43276872148327034), ('CustomBondForce', 1.6188575373098157)]


INFO:geometry:	final no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	there are 3 bond forces in the no-nonbonded final system
INFO:geometry:	there are 19 angle forces in the no-nonbonded final system
INFO:geometry:	there are 22 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 1.7890162426022216
INFO:geometry:total reduced energy added from growth system: -1.7818771939630165
INFO:geometry:final reduced energy 0.00713904863920553
INFO:geometry:sum of energies: 0.0071390486392051145
INFO:geometry:magnitude of difference in the energies: 4.440892098500626e-16
INFO:geometry:Final logp_proposal: 3.191355854995794


added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.0023465627603357813), ('CustomTorsionForce', 6.390202536223275e-05), ('CustomBondForce', -1.7842876587487142)]


In [4]:
htf = HybridTopologyFactory(molecule_setup._solvent_topology_proposal,
                            molecule_setup._ligand_positions_old_solvated,
                            molecule_setup._ligand_positions_new_solvated)

In [5]:
from perses.tests.utils import validate_endstate_energies
ENERGY_THRESHOLD = 1e-6

from openmmtools.constants import kB
temperature = 300 * unit.kelvin

In [6]:
zero_state_error, one_state_error = validate_endstate_energies(htf._topology_proposal, htf, molecule_setup._solvated_added_valence_energy, molecule_setup._solvated_subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD)

			HarmonicBondForce: 0.3302988765866683
			HarmonicAngleForce: 0.17577870393365402
			PeriodicTorsionForce: 1.2853491268675972
			NonbondedForce: -343.0873107062181
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-341.29588399883016
rp: -341.0191472827562
			CustomBondForce: 0.3302988765866683
			HarmonicBondForce: 0.0
			CustomAngleForce: 0.17343214117331823
			HarmonicAngleForce: 1.6146937954693068
			CustomTorsionForce: 1.285285224842235
			PeriodicTorsionForce: 0.43283262350863255
			NonbondedForce: -334.0811220767723
			CustomNonbondedForce: -7.387331092134983
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:-337.6319105073271
rp: -337.35517379125326
			CustomBondForce: 64.3572998501014
			HarmonicBondForce: 0.0
			CustomAngleForce: 0.7988136664411729
			HarmonicAngleForce: 1.6146937954693068
			CustomTorsionForce: 0.010659379893790519
			PeriodicTorsionForce: 0.43283262350863255
			NonbondedForce: -321.2413540615145
		

so it looks like we can properly maintain an endstate validation with small molecules, but what about protein mutations; is this the reason we are having issues with protein mutations?

anyway, at this point, let's just run this and see if it goes fast...

In [7]:
def create_splitting_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
#     integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
#                                                      timestep = 0.5* unit.femtoseconds,
#                                                      splitting = 'V R O R V',
#                                                      measure_shadow_work = False,
#                                                      measure_heat = False,
#                                                      constraint_tolerance = constraint_tol,
#                                                      collision_rate = 5.0 / unit.picoseconds)
    mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep = 4.0 * unit.femtoseconds,
                                                             collision_rate=5.0 / unit.picosecond,
                                                             n_steps=1,
                                                             reassign_velocities=False,
                                                             n_restart_attempts=20,
                                                             splitting="V R O R V",
                                                             constraint_tolerance=constraint_tol)
    
    
    #print(integrator_1.getConstraintTolerance())
    
#     fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
#     fast_sampler_state.apply_to_context(fast_context)
    
    return mcmc_moves, fast_thermodynamic_state, fast_sampler_state


    
def create_langevin_integrator(htf, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    fast_lambda_alchemical_state = RelativeAlchemicalState.from_system(htf.hybrid_system)
    fast_lambda_alchemical_state.set_alchemical_parameters(0.0, LambdaProtocol(functions = 'default'))
    
    fast_thermodynamic_state = CompoundThermodynamicState(ThermodynamicState(htf.hybrid_system, temperature = temperature),composable_states = [fast_lambda_alchemical_state])
    
    fast_sampler_state = SamplerState(positions = htf._hybrid_positions, box_vectors = htf.hybrid_system.getDefaultPeriodicBoxVectors())
    
    integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
                                                     timestep = 4.0* unit.femtoseconds,
                                                     splitting = 'V R O R V',
                                                     measure_shadow_work = False,
                                                     measure_heat = False,
                                                     constraint_tolerance = constraint_tol,
                                                     collision_rate = 5.0 / unit.picoseconds)
#     mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep = 4.0 * unit.femtoseconds,
#                                                              collision_rate=5.0 / unit.picosecond,
#                                                              n_steps=1,
#                                                              reassign_velocities=False,
#                                                              n_restart_attempts=20,
#                                                              splitting="V R R R O R R R V",
#                                                              constraint_tolerance=constraint_tol)
    
    
    #print(integrator_1.getConstraintTolerance())
    
    fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
    fast_sampler_state.apply_to_context(fast_context)
    
    return fast_context, fast_thermodynamic_state, fast_sampler_state, fast_integrator    
    
    

In [8]:
def splitting_integrator_timer(move, thermostate, sstate, num_steps = 10):
    import time
    _time = []
    move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        #integrator.step(1)
        move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

In [9]:
def langevin_integrator_timer(integrator, num_steps = 10):
    import time
    _time = []
    integrator.step(1)
    #move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        integrator.step(1)
        #move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

now we can actually time the two integration schemes!!!!

In [10]:
mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state = create_splitting_integrator(htf, constraint_tol = 1e-6)
context, langevin_thermodynamic_state, langevin_sampler_state, langevin_integrator   = create_langevin_integrator(htf, constraint_tol = 1e-6)

In [ ]:
splitting_time = splitting_integrator_timer(mcmc_moves, splitting_thermodynamic_state, splitting_sampler_state, num_steps = 1000)
langevin_time = langevin_integrator_timer(langevin_integrator, num_steps = 1000)

In [ ]:
print(f"splitting_time_integrator: {np.average(splitting_time)} +/- {np.std(splitting_time)}")
print(f"langevin time integrator: {np.average(langevin_time)} +/- {np.std(langevin_time)}")